In [43]:
from transformers import pipeline
#tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
#model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [74]:
import pandas as pd
df = pd.read_csv("comments.csv")
df.head()

,Comment
0,The fact that this hasn’t been made into meme ...
1,2:17 keep in mind
2,I know what you can do without a brain...\n\n\...
3,What can I do without a brain? Run for congres...
4,"Answer: nothing much, but I've seen people mak..."


In [75]:
df['Comment']=df['Comment'].astype(str)

In [76]:
import pandas as pd
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def remove_combining_chars(text):
    # Remove combining characters using the 'Mn' (Mark, Nonspacing) category
    return ''.join(c for c in text if unicodedata.category(c) != 'Mn')

# Define a function to clean the text
def clean_text(text):
    # Remove combining characters
    cleaned_text = remove_combining_chars(text)

    # Remove all non-alphanumeric characters
    cleaned_text = re.sub(r"[^a-zA-Z0-9\s]", "", cleaned_text)

    return cleaned_text

# Assuming your dataset has a column named 'text_column' containing the text entries
# Replace 'text_column' with the name of the column you want to clean
df['cleaned_text'] = df['Comment'].apply(clean_text)

In [77]:
def get_emotion_label(text):
      return(emotion(text)[0]['label'])
        
def get_score(text):
      return(emotion(text)[0]['score'])

In [78]:
stop_words = set(stopwords.words('english'))
def cleanTweet(tweetText):
    tweetText =re.sub(pattern='http\S+\s*', repl=' ', string=tweetText) # URLs
    tweetText = re.sub(pattern='RT|cc', repl=' ', string=tweetText)  # remove RT and cc
    tweetText = re.sub(pattern='@\S+', repl=' ', string=tweetText) # hashtags
    tweetText = re.sub('@S+', '  ', tweetText)  # mentions
    tweetText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), repl=' ', string=tweetText) # punctuations
    tweetText = re.sub(r'[^\x00-\x7f]',r' ', tweetText) # numbers
    tweetText = re.sub('\s+', ' ', tweetText) # extra whitespace
    tweetText = tweetText.lower()
    words = word_tokenize(tweetText)
    filtered_sentence = [w for w in words if not w in stop_words]
    tweetText = ' '.join(filtered_sentence)
    return tweetText

df['cleanedTweets'] = df['Comment'].apply(lambda tweet:cleanTweet(tweet))
df.head()

,Comment,cleaned_text,cleanedTweets
0,The fact that this hasn’t been made into meme ...,The fact that this hasnt been made into meme t...,fact made meme template yet astounding
1,2:17 keep in mind,217 keep in mind,2 17 keep mind
2,I know what you can do without a brain...\n\n\...,I know what you can do without a brain\n\n\n\n...,know without brain suport lgbtq
3,What can I do without a brain? Run for congres...,What can I do without a brain Run for congress...,without brain run congress president sort poli...
4,"Answer: nothing much, but I've seen people mak...",Answer nothing much but Ive seen people making...,answer nothing much seen people making tiktoks...


In [81]:
df = df.head(500)

In [82]:
df['emotion']=df['cleanedTweets'].apply(get_emotion_label)

C:\Users\younes G\AppData\Local\Temp\ipykernel_29940\78684679.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df['score']=df['Comment'].apply(get_score)

In [83]:
freq = df['emotion'].value_counts(ascending=False).rename_axis('unique_values').reset_index(name='counts')
freq = pd.DataFrame(freq)
freq.head()

,unique_values,counts
0,neutral,160
1,amusement,5
2,approval,5
3,love,5
4,surprise,4


In [84]:
import plotly.express as px
fig = px.pie(freq,values='counts', names='unique_values')
fig.show()

In [85]:
fig = px.bar( x=freq['unique_values'], y=freq['counts'], title='distribution of emotions') 
fig.show()

In [86]:
print("the video present an emotion of" +" "+ freq['unique_values'][0])

the video present an emotion of neutral
